## Load Data

In [ ]:
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import os
from torchvision.transforms import v2
from face_identifier.dataset import FaceDataset


augmentation = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.ColorJitter(brightness=.3, hue=.1),
    v2.RandomPhotometricDistort(p=0.2),
])


# paths
path = '/home/anthony/Documents/Homework Documents/Inno Lab Group Project/WFLW'
path_images = os.path.join(path, 'WFLW_images')
# path = '/home/anthony/Documents/Homework Documents/Inno Lab Group Project/Face Detection'
# path_images = path
training_set = FaceDataset(os.path.join(path, 'training.txt'), path_images, transform=augmentation)
validation_set = FaceDataset(os.path.join(path, 'validation.txt'), path_images)
test_set = FaceDataset(os.path.join(path, 'test.txt'), path_images)

# check image
for i in range(5):
    training_set.draw(i)

## Build Model

In [ ]:
import torch
from face_identifier.model import FaceDetector


model = FaceDetector()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = torch.nn.L1Loss()
model

## Train Model

In [ ]:
import os
from face_identifier.processes import ModelTrainer, test

save_path = 'face-detector.pt'
if os.path.exists(save_path):
    model.load_state_dict(torch.load(save_path))
trainer = ModelTrainer(model, training_set, validation_set, optimizer, loss_func=loss_func, device=device, batch_size=64,
                       save_path=save_path, log_name="Detect Loss")
trainer.train(10)

In [ ]:
trainer.train(50)

In [ ]:
print(f'Best training loss: {trainer.best_loss:.5f}')

## Test & Visualization

In [ ]:
# load the best model
# torch.save(model.state_dict(), 'face-detector-last.pt')
model.load_state_dict(torch.load('face-detector.pt'))

In [ ]:
from torch.utils.data import DataLoader
from face_identifier.processes import test


test_loader = DataLoader(test_set, batch_size=64)
test_loss = test(model, test_loader, loss_func, device=device)
print(f'Test loss: {test_loss:.5f}')

In [ ]:
with torch.no_grad():
    model.to("cpu")
    model.eval()
    for i in range(5, 10):
        image, _, _ = test_set[i]
        image = image.view(1, *image.shape)
        prediction = model(image)[0]
        ax = test_set.draw(i, prediction)
        ax.set_title(f'Has Face: {prediction[0].item():.5f}')